In [2]:
import pandas as pd
from wmfdata import hive

# Daily data

In [ ]:
CREATE TABLE IF NOT EXISTS jiawang2.content_edit_daily (
`date` DATE,
project string,
wiki_db string,
topic string,
main_topic string,
sub_topic string,
user_is_anonymous boolean ,
user_is_bot boolean,
edit_count BIGINT
) 
PARTITIONED BY (month string)
STORED AS RCFILE



## Extract daily data for 2021-06 ~ 2021-07
use new topic table: isaacj.article_topics_outlinks_2021_07

In [5]:
query_daily_content_edit_2021_07='''
INSERT INTO jiawang2.content_edit_daily
PARTITION(month= '{MONTH_STR}')

SELECT DATE_FORMAT(event_timestamp, 'yyyy-MM-dd') AS  `date`, 
        SUBSTR(w.domain_name, 0, LENGTH(w.domain_name) - 4) AS project,
        h.wiki_db, ato.topic,
        tc.main_topic,tc.sub_topic,
        event_user_is_anonymous AS user_is_anonymous,
        SIZE(event_user_is_bot_by_historical) > 0 AS user_is_bot,
        COUNT(1) AS edit_count
FROM wmf.mediawiki_history AS h
INNER JOIN canonical_data.wikis w ON h.wiki_db = w.database_code AND
    w.database_group = "wikipedia"
INNER JOIN isaacj.article_topics_outlinks_2021_07 ato ON (
  w.database_code =  ato.wiki_db
  AND h.page_id = ato.pageid
  AND ato.score >= 0.5
)
LEFT JOIN cchen.topic_component tc ON ato.topic = tc.topic  
WHERE snapshot = '{SNAPSHOT}'
      AND SUBSTR(event_timestamp, 1, 7) = '{MONTH_STR}'
      AND event_entity = 'revision'
      AND event_type = 'create' 
      AND page_namespace = 0
GROUP BY DATE_FORMAT(event_timestamp, 'yyyy-MM-dd'), 
SUBSTR(w.domain_name, 0, LENGTH(w.domain_name) - 4), w.database_code,  
  h.wiki_db, ato.topic,
  tc.main_topic,tc.sub_topic,
  event_user_is_anonymous, SIZE(event_user_is_bot_by_historical) > 0
'''

In [3]:
month_list = pd.date_range(start='2021-06-01', end='2021-07-01', freq='1MS')

In [4]:
month_list

DatetimeIndex(['2021-06-01', '2021-07-01'], dtype='datetime64[ns]', freq='MS')

In [6]:
for m in month_list:
    month_string= m.strftime("%Y-%m")
    print(month_string)
    hive.run(query_daily_content_edit_2021_07.format(MONTH_STR=month_string, SNAPSHOT='2021-07'))

2021-06
2021-07
